In [ ]:
import pandas as pd
from scipy import stats

In [ ]:
df_AB = pd.read_excel("/content/Artea - 521703-XLS-ENG.xlsx", sheet_name = "AB_test")

In [ ]:
df_AB.head()

,id,trans_after,revenue_after,test_coupon,channel_acq,num_past_purch,spent_last_purchase,weeks_since_visit,browsing_minutes,shopping_cart
0,AB_1,0,0.0,0,2,6,62.99,6,1,0
1,AB_2,0,0.0,1,1,2,53.99,0,7,1
2,AB_3,0,0.0,1,2,3,88.98,3,4,0
3,AB_4,0,0.0,0,2,1,68.99,1,19,0
4,AB_5,0,0.0,1,3,3,66.49,4,20,0


In [ ]:
## Testing Revenue after experiment


df1 = df_AB.copy()

coupon_revenue = df1[df1["test_coupon"] == 1 ]["revenue_after"]
no_coupon_revenue = df1[df1["test_coupon"] == 0]["revenue_after"]


# Perform two-sample t-test
t_statistic, p_value = stats.ttest_ind(coupon_revenue, no_coupon_revenue, alternative='less')
p_value

0.3589326089162478

In [ ]:
## Testing transactions after experiment

df1 = df_AB.copy()

coupon_trans = df1[df1["test_coupon"] == 1 ]["trans_after"]
no_coupon_trans = df1[df1["test_coupon"] == 0]["trans_after"]


# Perform two-sample t-test
t_statistic, p_value = stats.ttest_ind(coupon_trans, no_coupon_trans, alternative='greater')
p_value

0.013673094150031226

In [ ]:
## Since overall there are no significant increse in the revenue after the coupons which is evident because individuals with coupons purchased after
# discount.



channels = {1: "Google", 2: "Facebook", 3: "Instagram",4:"Referal", 5 : "Other"}

for channel in df1["channel_acq"].unique():
  filtered_df = df1[df1["channel_acq"] == channel]

  print("CHANNEL : " + channels[channel] + "\n")

  target_cols = ["trans_after","revenue_after"]


  for tc in target_cols:
    coupon = filtered_df[filtered_df["test_coupon"] == 1][tc]
    non_coupon = filtered_df[filtered_df["test_coupon"] == 0][tc]

    print("NULL HYPOTHESIS : "+tc+" for individuals receiving coupons is less than or equal to individuals with no coupons")
    print("ALT HYPOTHESIS : "+tc+" for individuals receiving coupons is greater to individuals with no coupons")

    t_statistic, p_value = stats.ttest_ind(coupon, non_coupon, alternative='greater')
    print(p_value)
    if (p_value < 0.05):
      print("TEST RESULTS SIGNIFICANT \n")
    else:
      print("NULL HYPOTHESIS ACCEPTED \n")

CHANNEL : Facebook

NULL HYPOTHESIS : trans_after for individuals receiving coupons is less than or equal to individuals with no coupons
ALT HYPOTHESIS : trans_after for individuals receiving coupons is greater to individuals with no coupons
0.0210907638005385
TEST RESULTS SIGNIFICANT 

NULL HYPOTHESIS : revenue_after for individuals receiving coupons is less than or equal to individuals with no coupons
ALT HYPOTHESIS : revenue_after for individuals receiving coupons is greater to individuals with no coupons
0.2616575483513298
NULL HYPOTHESIS ACCEPTED 

CHANNEL : Google

NULL HYPOTHESIS : trans_after for individuals receiving coupons is less than or equal to individuals with no coupons
ALT HYPOTHESIS : trans_after for individuals receiving coupons is greater to individuals with no coupons
0.943509799064446
NULL HYPOTHESIS ACCEPTED 

NULL HYPOTHESIS : revenue_after for individuals receiving coupons is less than or equal to individuals with no coupons
ALT HYPOTHESIS : revenue_after for i

We have identified that Facebook and Instagram channel users are showing a significant increase in number of transactions.

In [ ]:
df1.head()

,id,trans_after,revenue_after,test_coupon,channel_acq,num_past_purch,spent_last_purchase,weeks_since_visit,browsing_minutes,shopping_cart
0,AB_1,0,0.0,0,2,6,62.99,6,1,0
1,AB_2,0,0.0,1,1,2,53.99,0,7,1
2,AB_3,0,0.0,1,2,3,88.98,3,4,0
3,AB_4,0,0.0,0,2,1,68.99,1,19,0
4,AB_5,0,0.0,1,3,3,66.49,4,20,0


In [ ]:
df1 = df_AB.copy()
df1.drop(["id"],axis =1 , inplace = True)
df1["channel_acq"] = df1["channel_acq"].astype(str)
# Convert the categorical column to dummy variables
dummy_df = pd.get_dummies(df1['channel_acq'], drop_first=True, prefix='channel_acq')
df1.drop(['channel_acq'],axis = 1, inplace =True)
# Concatenate the dummy variables with the original dataframe
df1 = pd.concat([df1, dummy_df], axis=1)
did_transact = lambda x : 1 if x > 0 else 0
df1["did_trasnact"] = df1["trans_after"].apply(did_transact)
df1.drop(["trans_after"],axis = 1, inplace =True)
df1.head()

,revenue_after,test_coupon,num_past_purch,spent_last_purchase,weeks_since_visit,browsing_minutes,shopping_cart,channel_acq_2,channel_acq_3,channel_acq_4,channel_acq_5,did_trasnact
0,0.0,0,6,62.99,6,1,0,1,0,0,0,0
1,0.0,1,2,53.99,0,7,1,0,0,0,0,0
2,0.0,1,3,88.98,3,4,0,1,0,0,0,0
3,0.0,0,1,68.99,1,19,0,1,0,0,0,0
4,0.0,1,3,66.49,4,20,0,0,1,0,0,0


In [ ]:
import statsmodels.api as sm

# Separate the target variable and the features
target = df1['did_trasnact']
features = df1.drop(['did_trasnact'], axis=1)

# Add intercept to the features
features = sm.add_constant(features)

# Fit logistic regression model
logit_model = sm.Logit(target, features)
result = logit_model.fit()

# Display model summary
print(result.summary())

In [ ]:
df1.head()

,revenue_after,test_coupon,num_past_purch,spent_last_purchase,weeks_since_visit,browsing_minutes,shopping_cart,channel_acq_2,channel_acq_3,channel_acq_4,channel_acq_5,did_trasnact
0,0.0,0,6,62.99,6,1,0,1,0,0,0,0
1,0.0,1,2,53.99,0,7,1,0,0,0,0,0
2,0.0,1,3,88.98,3,4,0,1,0,0,0,0
3,0.0,0,1,68.99,1,19,0,1,0,0,0,0
4,0.0,1,3,66.49,4,20,0,0,1,0,0,0


In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
features = df1.copy()
vif_data = pd.DataFrame()
vif_data["Variable"] = features.columns
vif_data["VIF"] = [variance_inflation_factor(features.values, i) for i in range(features.shape[1])]

print(vif_data)

               Variable       VIF
0         revenue_after  5.875275
1           test_coupon  1.850584
2        num_past_purch  2.200470
3   spent_last_purchase  2.388869
4     weeks_since_visit  2.531271
5      browsing_minutes  3.177310
6         shopping_cart  1.384494
7         channel_acq_2  1.411163
8         channel_acq_3  1.691619
9         channel_acq_4  1.092039
10        channel_acq_5  1.050892
11         did_trasnact  5.910108


In [ ]:
model_df = df1.drop("revenue_after", axis = 1)
model_df.columns

Index(['test_coupon', 'num_past_purch', 'spent_last_purchase',
       'weeks_since_visit', 'browsing_minutes', 'shopping_cart',
       'channel_acq_2', 'channel_acq_3', 'channel_acq_4', 'channel_acq_5',
       'did_trasnact'],
      dtype='object')

In [ ]:
import statsmodels.api as sm

# Separate the target variable and the features
target = df1['did_trasnact']
features = df1.drop(['did_trasnact',"revenue_after"], axis=1)

# Add intercept to the features
features = sm.add_constant(features)

# Fit logistic regression model
logit_model = sm.Logit(target, features)
result = logit_model.fit()

# Display model summary
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.285116
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:           did_trasnact   No. Observations:                 5000
Model:                          Logit   Df Residuals:                     4989
Method:                           MLE   Df Model:                           10
Date:                Thu, 17 Aug 2023   Pseudo R-squ.:                  0.2028
Time:                        19:01:14   Log-Likelihood:                -1425.6
converged:                       True   LL-Null:                       -1788.3
Covariance Type:            nonrobust   LLR p-value:                2.177e-149
                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                  -4.2983      0.197    -21.822      0.000      -4.684      -3.912
te

In [ ]:
## FINAL LOGISTIC REGRESSION MODEL





import statsmodels.api as sm

# Separate the target variable and the features
target = df1['did_trasnact']
features = df1.drop(['did_trasnact',"revenue_after","spent_last_purchase"], axis=1)

# Add intercept to the features
features = sm.add_constant(features)

# Fit logistic regression model
logit_model = sm.Logit(target, features)
result_logit = logit_model.fit()

# Display model summary
print(result_logit.summary())

Optimization terminated successfully.
         Current function value: 0.285143
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:           did_trasnact   No. Observations:                 5000
Model:                          Logit   Df Residuals:                     4990
Method:                           MLE   Df Model:                            9
Date:                Thu, 17 Aug 2023   Pseudo R-squ.:                  0.2027
Time:                        19:02:17   Log-Likelihood:                -1425.7
converged:                       True   LL-Null:                       -1788.3
Covariance Type:            nonrobust   LLR p-value:                2.703e-150
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                -4.2735      0.191    -22.359      0.000      -4.648      -3.899
test_cou

In [ ]:

df2 = df_AB.copy()

In [ ]:
df2 = df_AB.copy()
df2.drop(["id"],axis =1 , inplace = True)
df2["channel_acq"] = df2["channel_acq"].astype(str)
# Convert the categorical column to dummy variables
dummy_df = pd.get_dummies(df2['channel_acq'], drop_first=True, prefix='channel_acq')
df2.drop(['channel_acq'],axis = 1, inplace =True)
# Concatenate the dummy variables with the original dataframe
df2 = pd.concat([df2, dummy_df], axis=1)
df2.head()

,trans_after,revenue_after,test_coupon,num_past_purch,spent_last_purchase,weeks_since_visit,browsing_minutes,shopping_cart,channel_acq_2,channel_acq_3,channel_acq_4,channel_acq_5
0,0,0.0,0,6,62.99,6,1,0,1,0,0,0
1,0,0.0,1,2,53.99,0,7,1,0,0,0,0
2,0,0.0,1,3,88.98,3,4,0,1,0,0,0
3,0,0.0,0,1,68.99,1,19,0,1,0,0,0
4,0,0.0,1,3,66.49,4,20,0,0,1,0,0


In [ ]:
#FINAL MODEL FOR LINEAR REGRESSION TRANSACTIONS



# Separate the target variable and the features
target = df2['trans_after']
features = df2.drop(['trans_after',"revenue_after"], axis=1)

# Add intercept to the features
features = sm.add_constant(features)

# Fit linear regression model
model_linear = sm.OLS(target, features)
result_linear = model_linear.fit()

# Display model summary
print(result_linear.summary())

                            OLS Regression Results                            
Dep. Variable:            trans_after   R-squared:                       0.177
Model:                            OLS   Adj. R-squared:                  0.176
Method:                 Least Squares   F-statistic:                     107.7
Date:                Thu, 17 Aug 2023   Prob (F-statistic):          3.35e-203
Time:                        19:25:11   Log-Likelihood:                -2262.1
No. Observations:                5000   AIC:                             4546.
Df Residuals:                    4989   BIC:                             4618.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                  -0.0649    

In [ ]:


#NOW LETS PREDICT IN THE NEXT CAMPAIGN DATASET
df_next = pd.read_excel("/content/Artea - 521703-XLS-ENG.xlsx", sheet_name = "Next_Campaign")

In [ ]:
df3 = df_next.copy()

#putting 1 in all test_coupon columns
df3["test_coupon"] = [1] * df3.shape[0]

df3.drop(["id"],axis =1 , inplace = True)
df3["channel_acq"] = df3["channel_acq"].astype(str)
# Convert the categorical column to dummy variables
dummy_df = pd.get_dummies(df3['channel_acq'], drop_first=True, prefix='channel_acq')
df3.drop(['channel_acq'],axis = 1, inplace =True)
# Concatenate the dummy variables with the original dataframe
df3 = pd.concat([df3, dummy_df], axis=1)
df3.head()

,num_past_purch,spent_last_purchase,weeks_since_visit,browsing_minutes,shopping_cart,test_coupon,channel_acq_2,channel_acq_3,channel_acq_4,channel_acq_5
0,0,0.00,0,16,1,1,0,0,0,0
1,0,0.00,2,19,0,1,0,1,0,0
2,5,63.99,1,14,0,1,0,0,1,0
3,0,0.00,7,15,0,1,0,0,0,0
4,2,52.49,2,19,0,1,1,0,0,0


In [ ]:
# Select only the required columns for prediction
required_columns = ['test_coupon', 'num_past_purch','weeks_since_visit',
                    'browsing_minutes', 'shopping_cart', 'channel_acq_2',
                    'channel_acq_3', 'channel_acq_4', 'channel_acq_5']

new_data_selected = df3[required_columns]

# Add intercept to the new dataset
new_column = pd.Series([1.0]*6000, name='const')
new_data_selected.insert( 0, "const", new_column)

# Use the trained model to make predictions
predicted_probs = result_logit.predict(new_data_selected)  # 'result_logit' is the trained logistic regression model

# If you want to convert predicted probabilities to binary predictions (0 or 1)
predicted_labels = (predicted_probs >= 0.7).astype(int)

# Print the predicted probabilities or labels
print(predicted_probs)
print(predicted_labels)

0       0.104885
1       0.067566
2       0.286599
3       0.007682
4       0.117363
          ...   
5995    0.034423
5996    0.010381
5997    0.264881
5998    0.049415
5999    0.227979
Length: 6000, dtype: float64
0       0
1       0
2       0
3       0
4       0
       ..
5995    0
5996    0
5997    0
5998    0
5999    0
Length: 6000, dtype: int64


In [ ]:
new_data_selected

,const,test_coupon,num_past_purch,weeks_since_visit,browsing_minutes,shopping_cart,channel_acq_2,channel_acq_3,channel_acq_4,channel_acq_5
0,1.0,1,0,0,16,1,0,0,0,0
1,1.0,1,0,2,19,0,0,1,0,0
2,1.0,1,5,1,14,0,0,0,1,0
3,1.0,1,0,7,15,0,0,0,0,0
4,1.0,1,2,2,19,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...
5995,1.0,1,0,4,2,1,0,0,0,0
5996,1.0,1,0,5,13,0,0,0,0,0
5997,1.0,1,2,1,1,1,1,0,0,0
5998,1.0,1,1,6,22,0,1,0,0,0


In [ ]:
df_next["Predictions"] = predicted_labels

In [ ]:
df_next_positive = df_next[df_next["Predictions"] == 1]

In [ ]:
df_next_positive.shape

(66, 8)

In [ ]:
df_next.shape

(6000, 8)

In [ ]:
df4 = df_next_positive.copy()
df4.drop(["id"],axis =1 , inplace = True)
df4["channel_acq"] = df4["channel_acq"].astype(str)
# Convert the categorical column to dummy variables
dummy_df = pd.get_dummies(df4['channel_acq'], drop_first=True, prefix='channel_acq')
df4["test_coupon"] = df4["Predictions"]
df4.drop(['channel_acq',"Predictions"],axis = 1, inplace =True)
# Concatenate the dummy variables with the original dataframe
df4 = pd.concat([df4, dummy_df], axis=1)
df4.head()

,num_past_purch,spent_last_purchase,weeks_since_visit,browsing_minutes,shopping_cart,test_coupon,channel_acq_2,channel_acq_3,channel_acq_4,channel_acq_5
13,10,63.99,0,21,1,1,1,0,0,0
34,13,57.99,3,23,0,1,0,1,0,0
53,12,170.97,2,14,1,1,0,1,0,0
192,11,59.99,1,23,0,1,0,1,0,0
241,9,70.49,1,19,1,1,0,1,0,0


In [ ]:
# Select only the required columns for prediction
required_columns = ['test_coupon', 'num_past_purch', 'spent_last_purchase',"browsing_minutes" ,'weeks_since_visit', 'shopping_cart', 'channel_acq_2','channel_acq_3','channel_acq_4', 'channel_acq_5']
new_data_selected = df4[required_columns]
new_data_selected.reset_index(inplace =True)
new_data_selected.drop("index", axis = 1, inplace = True)
# coefficients = [-0.0649 ,0.0256,0.0571,-0.0004,-0.0204,0.0031,0.1728,0.1131,0.1083,0.1295,0.1587]


# Add intercept to the new dataset
new_column = pd.Series([1.0]*df4.shape[0], name='const')
new_data_selected.insert( 0, "const", new_column)

# predictions = new_data_selected.dot(coefficients)
# Use the trained model to make predictions
predicted_values = result_linear.predict(new_data_selected)  # 'result_linear' is the trained linear regression model

# Print the predicted values
print(predicted_values)

0     0.365849
1     0.330266
2     0.584915
3     0.209154
4     0.345458
        ...   
61    0.241127
62    0.599706
63    0.186378
64    0.510421
65    0.386514
Length: 66, dtype: float64


In [ ]:
# Calculate the sum of predicted values
sum_of_predicted_values = predicted_values.sum()

# Print the sum
print("Sum of Predicted Values:", sum_of_predicted_values)

Sum of Predicted Values: 33.107179345066704


In [ ]:
new_data_selected

,const,index,test_coupon,num_past_purch,spent_last_purchase,browsing_minutes,weeks_since_visit,shopping_cart,channel_acq_2,channel_acq_3,channel_acq_4,channel_acq_5
0,1.0,13,1,10,63.99,21,0,1,1,0,0,0
1,1.0,34,1,13,57.99,23,3,0,0,1,0,0
2,1.0,53,1,12,170.97,14,2,1,0,1,0,0
3,1.0,192,1,11,59.99,23,1,0,0,1,0,0
4,1.0,241,1,9,70.49,19,1,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
61,1.0,5423,1,11,83.99,21,1,0,0,1,0,0
62,1.0,5495,1,19,51.49,22,7,0,0,0,0,0
63,1.0,5576,1,12,221.97,24,1,0,0,1,0,0
64,1.0,5644,1,12,61.49,11,1,0,0,1,0,0


In [ ]:
new_data_selected.to_csv("new_data_selected.csv")

In [ ]:
new_data_selected["spent_last_purchase"].sum()

5574.589999999999

In [ ]:
df_demo = pd.read_excel('/content/Artea - 521704-XLS-ENG_DEMOGRAPHICS.xlsx', sheet_name = "Next_Campaign_demog")

In [ ]:
df_demo.head()

,id,minority,non_male,channel_acq,num_past_purch,spent_last_purchase,weeks_since_visit,browsing_minutes,shopping_cart
0,Next_1,1,0,1,0,0.00,0,16,1
1,Next_2,0,1,3,0,0.00,2,19,0
2,Next_3,1,0,4,5,63.99,1,14,0
3,Next_4,1,0,1,0,0.00,7,15,0
4,Next_5,0,1,2,2,52.49,2,19,0


In [ ]:
df_demo["Predictions"] = predicted_labels

In [ ]:
from scipy.stats import chi2_contingency

contingency_table = pd.crosstab(df_demo['minority'], df_demo['Predictions'])

# Perform the chi-squared test
chi2, p, dof, expected = chi2_contingency(contingency_table)

# Print the test results
print("Chi-Squared:", chi2)
print("P-Value:", p)
print("Degrees of Freedom:", dof)
print("Expected Frequencies:", expected)

Chi-Squared: 7.905096590369771
P-Value: 0.004929570931501208
Degrees of Freedom: 1
Expected Frequencies: [[4700.717   52.283]
 [1233.283   13.717]]


In [ ]:
contingency_table

Predictions,0,1
minority,,
0,4691,62
1,1243,4


In [ ]:
from scipy.stats import chi2_contingency

contingency_table = pd.crosstab(df_demo['non_male'], df_demo['Predictions'])

# Perform the chi-squared test
chi2, p, dof, expected = chi2_contingency(contingency_table)

# Print the test results
print("Chi-Squared:", chi2)
print("P-Value:", p)
print("Degrees of Freedom:", dof)
print("Expected Frequencies:", expected)

Chi-Squared: 4.227597272489458
P-Value: 0.039771705057737944
Degrees of Freedom: 1
Expected Frequencies: [[2490.302   27.698]
 [3443.698   38.302]]


In [ ]:
contingency_table

Predictions,0,1
non_male,,
0,2499,19
1,3435,47


In [ ]:
## CLEARLY MODEL IS BIASED TOWARDS FEMALE POPULATION and non_minority